## Postal code Analisys

#### Satislar dosyası düzenlemeler

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")
warnings.warn("thisnwill not show")

In [2]:
# Satışlar datasını df olarak atıyoruz.
df = pd.read_excel(r"C:\Users\Admin\Desktop\EDA The Rug\orjinal datalar\Satislar.xlsx")

In [3]:
# ortak satırlardaki Nan değerleri siliyoruz.
df.dropna(subset = "cPostCode", inplace=True)

In [4]:
# duplicated valueları belirleyip siliyoruz.
df.duplicated().value_counts()

False    94871
True       101
dtype: int64

In [5]:
df.drop_duplicates(inplace=True)      

In [6]:
# posta kodları ve ülke isimlerinden oluşan yeni dataframe i oluşturuyoruz.
satislar_pcode = pd.DataFrame(df[["cPostCode", "Country"]])

In [7]:
# Sütun isimlerini coğrafi özelliklerin olduğu data ile birleştirmek için sütun ismini değiştiriyoruz.
satislar_pcode.rename(columns ={"cPostCode": "Postcode"}, inplace = True)

In [8]:
# posta kodu sütununda Redacted yazan satırları siliyoruz.
satislar_pcode = satislar_pcode[satislar_pcode.Postcode != "Redacted"]

In [9]:
# Datamızda UK dışında dikkate alınacak bir satış olmadığı için bu ülkelerin posta kodlarını datamızdan çıkarıyoruz.
satislar_pcode.groupby("Country").Postcode.count()

Country
Austria               2
France                1
Germany              62
Ireland              42
Netherlands           3
Romania               2
United Kingdom    39086
United States         1
Name: Postcode, dtype: int64

In [10]:
satislar_pcode = satislar_pcode[satislar_pcode.Country == "United Kingdom"]

In [11]:
satislar_pcode.groupby("Country").Postcode.count()

Country
United Kingdom    39086
Name: Postcode, dtype: int64

In [12]:
# Posta kodları eşleşmelerinde daha fazla başarı elde edebilmek için bizm datamızdaki küçük harfle yazılmış posta kodlarını büyük harfe çeviriyoruz.
satislar_pcode["Postcode"]= satislar_pcode.apply((lambda x: x["Postcode"].upper() if "United Kingdom" in x["Country"] else x["Postcode"]), axis=1)

In [13]:
(satislar_pcode['Postcode'].str.contains(' ') == False).sum() 
# Posta kodlarında son 3 basamaktan önce boşluk olması gerekiyor eğer bu boşluk yoksa hatalı oluyor.
# Burada da görüldüğü üzere 2657 adet posta kodu boşluksuz olarak yazılmış.

2657

In [14]:
# Posta kodlarındaki bütün boşlukları kaldırıp daha sonra olması gerektiği gibi -4.indexe boşluk atıyoruz.
satislar_pcode.Postcode = satislar_pcode.Postcode.str.replace(" ", "")

In [15]:
(satislar_pcode['Postcode'].str.contains(' ') == False).sum()

39086

In [16]:
# Şimdi bütün datamızı olması gerektiği formata sokuyoruz.
mask = satislar_pcode['Postcode'].str.contains(' ') == False
satislar_pcode.loc[mask, 'Postcode'] = satislar_pcode.loc[mask, 'Postcode'].str.slice(0, -3) + ' ' + satislar_pcode.loc[mask, 'Postcode'].str.slice(-3)

In [17]:
(satislar_pcode['Postcode'].str.contains(' ') == False).sum()

0

In [18]:
satislar_pcode.Postcode

2         OL13 8BT
28         DA1 5WW
30          W3 6TD
32        BD23 2NP
34        NE13 7FJ
            ...   
190022    BH15 4NF
190024    DA13 9EF
190026    EX24 6RY
190028     CV6 4BU
190030    DE22 2FJ
Name: Postcode, Length: 39086, dtype: object

In [20]:
# Bizim datamızda Country UK olarak geçiyor ancak diğer datadı daha detaylı yani England, wales gibi. Dolayısıyla birleştirme işlemi için Country sütununa 
# ihtiyaç olmayacak.

satislar_pcode.drop("Country", axis = 1 , inplace=True)

#### Ingilerenin posta kodlarına göre çeşitli bilgiler içeren veri

## Postcode CSV fields
####This page describes the meaning of the columns in the CSVs files available on the postcode downloads page. The main postcode CSV file contains the columns described below. Other CSV downloads generally contain a subset of these columns

* ``Postcode``	The Postcode that is the subject of this row in the format as it would be used in an address i.e. SE1 2AB
* ``In Use?``	Yes/No Flag to indicate that this Postcode is currently in use. Refer to columns [Introduced] and [Terminated] for further information
* ``Latitude``	Latitude of centroid of the Postcode for this row in decimal format i.e. 51.50205
* ``Longitude``	Longitude of centroid of the Postcode for this row in decimal format. i.e -0.07864. Negative values are those to the West of the zero (Greenwich) meridian
* ``Easting``	Co-ordinate of the East-West plane on the British National Grid. Reference
* ``Northing``	Co-ordinate of the North-South plane on the British National Grid
* ``Grid Ref``	Ordnance Survey grid reference in format xxnnnnnn i.e. TQ365751. Reference
* ``County``	Name of the Ceremonial County that this Postcode sits within. Some of these counties map directly to County Councils. Part of the hierarchy of: County->District->Ward
* ``District``	Name of the Local authority district (LAD)/unitary authority (UA)/ metropolitan district (MD)/ London borough (LB)/ council area (CA)/district council area (DCA) that this Postcode sits within. Part of the hierarchy of: County->District->Ward
* ``Ward``	Name of the local authority Ward that this Postcode sits within. Part of the hierarchy of: County->District->Ward
* ``District`` Code	Code of the District that this Postcode sits within. Part of the hierarchy of: County->District->Ward
* ``Ward Code``	Code of the Ward that this Postcode sits within. Part of the hierarchy of: County->District->Ward
* ``Country``	Name of the Country that this Postcode sits within. Values are: England, Scotland, Wales, Northern Ireland
* ``County Code``	Code of the County that this Postcode sits within. Part of the hierarchy of: County->District->Ward
* ``Constituency``	Name of the Paliamentary Constituency that this Postcode sits within
* ``Introduced``	Date that this Postcode was introduced
* ``Terminated``	Date that this Postcode was withdrawn from use (empty when the postcode is still in use)
* ``Parish``	Name of the Parish this this Postcode sits within. n.b Not all areas are designated a Parish
* ``National Park``	Name of the National Park that this Postcode sits within, where applicable
* ``Population``	Population of the area covered by the Postcode (from the 2011 census)
* ``Households``	Number of Households in the area covered by the Postcode (from the 2011 census)
* ``Built up area``	Name of the built-up area this postcode is part of
* ``Built up sub-division``	Name of the built-up area sub-division this postcode is part of
* ``Lower layer super output area``	Name of the Lower Layer Super Output Area (MSOA) that this Postcode sits within. Refer to: [LSOA Code]
* ``Rural/urban``	Description of the area covered by the Postcode i.e. Urban major conurbation, Remote small town, Urban city and town etc.
* ``Region``	Broad region of the UK i.e. London, South West, West Midlands, Yorkshire and The Humber etc. Used as ITL/NUTS level 1
* ``Altitude``	Height above sea level measured in Metres
* ``London zone``	Transport for London (TfL) Travel Zone indicator (London area only)
* ``LSOA Code``	Code for the Lower Layer Super Output Area (LSOA) that this Postcode sits within. Reference
* ``Local`` authority	County Council area that this Postcode sits within. For Unitary Authorties, London etc this value will not be populated. Reference
* ``MSOA Code Code for the Middle Layer`` Super Output Area (MSOA) that this Postcode sits within. Reference
* ``Middle layer super output area``	Name of the Middle Layer Super Output Area (MSOA) that this Postcode sits within. Refer to: [MSOA Code]
* ``Parish Code``	Code of the Parish that this Postcode sits within. n.b Not all areas are designated a Parish. e.g. E43000213
* ``Census output area``	Code for the Census Output Area e.g. E00016581
* ``Constituency Code``	Code of the Paliamentary Constituency that this Postcode sits within. e.g. E14000789
* ``Index of Multiple Deprivation``	The deprivation rank for the postcode, where 1 is the most deprived. The range of values is as follows
    - 1 - 32844 = England
    - 1 - 1909 = Wales
    - 1 - 6976 = Scotland
    - 1 - 890 = Northern Ireland
* ``Quality``	A Data Quality Flag for the current row. Values are as follows:
    - 1 = within the building of the matched address closest to the postcode mean.
    - 2 = as for status value 1, except by visual inspection of Landline maps (Scotland only).
    - 3 = approximate to within 50 metres.
    - 4 = postcode unit mean (mean of matched addresses with the same postcode, but not snapped to a building).
    - 5 = imputed by ONS, by reference to surrounding postcode grid references.
    - 6 = postcode sector mean, (mainly PO Boxes).
    - 7 = deliberately left blank
    - 8 = postcode terminated prior to Gridlink initiative, last known ONS postcode grid reference.
    - 9 = no grid reference available.
* ``User Type``	Shows whether the postcode is a small or large user.
    - 0 = small user (addresses receive less than 25 items of mail per day)
    - 1 = large user (addresses receive more than 25 items of mail per day)
* ``Last Updated``	The date the data for this postcode was last updated. The data is refreshed every 3 months when the ONS release the postcode directory
* ``Nearest station``	The nearest train station to the postcode. For London, also includes Underground and tram stops
* ``Distance to station``	The distance in kilometres to the nearest station from the postcode
* ``Postcode area``	The area of the postcode (e.g. KT)
* ``Postcode district``	The district of the the postcode (e.g. KT1)
* ``Police force``	The name of the police force responsible for this postcode
* ``Water company``	The name of the water company responsible for this postcode
* ``Plus Code``	Plus Code of the postcode
* ``Average income``	Average household income of the MSOA that the postcode is located in
* ``Sewage company``	If the sewage company is different from the water company, this column will contain the name of the sewage company
* ``Travel to work area``	The area in which the bulk of the population work
* ``ITL level 2``	ITL (formerly NUTS) is a system of geograhical sub-divisions defined by the EU. There are 3 levels, the Region column defines level 1. ITL is the post-Brexit name, although the regions currently remain the same
* ``ITL level 3``	As above
* ``UPRNs``	Comma separated list of Unique Property Reference Numbers within the postcode
* ``Distance to sea``	The approximate distance from this postcode to the sea in kilometres


In [21]:
post_uk = pd.read_csv("postcodes.csv")

In [22]:
post_uk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2670704 entries, 0 to 2670703
Data columns (total 53 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   Postcode                        object 
 1   In Use?                         object 
 2   Latitude                        float64
 3   Longitude                       float64
 4   Easting                         float64
 5   Northing                        float64
 6   Grid Ref                        object 
 7   County                          object 
 8   District                        object 
 9   Ward                            object 
 10  District Code                   object 
 11  Ward Code                       object 
 12  Country                         object 
 13  County Code                     object 
 14  Constituency                    object 
 15  Introduced                      object 
 16  Terminated                      object 
 17  Parish                     

In [23]:
# Data da bizim işimize yaramayacağını düşündüğüm verileri çıkarıyorum.

post_uk.drop(["London zone", "National Park", "Easting", "Northing", "Parish", "Population", "Built up area",
          "Built up sub-division", "Altitude", "LSOA Code", "Local authority", "MSOA Code",
           "Parish Code", "Census output area", "Index of Multiple Deprivation", "Distance to station",
           "Nearest station", "Water company", "Plus Code", "Sewage Company", "UPRNs", "Distance to sea", "Terminated"
          ], axis = 1, inplace = True)

In [24]:
post_uk.shape

(2670704, 30)

In [25]:
# Posta kodları olması gerektiği gibi düzgün
(post_uk['Postcode'].str.contains(' ') == False).sum()

0

In [26]:
# iki datayı eşleştiriyoruz.
merged_post_code = pd.merge(satislar_pcode, post_uk, on="Postcode", how='left')

In [27]:
merged_post_code.shape

(39086, 30)

In [28]:
merged_post_code.isnull().sum()

Postcode                             0
In Use?                            104
Latitude                           104
Longitude                          104
Grid Ref                           105
County                            3621
District                           105
Ward                               105
District Code                      105
Ward Code                          105
Country                            104
County Code                        105
Constituency                       105
Introduced                         104
Households                        3900
Lower layer super output area      105
Rural/urban                        553
Region                             104
Middle layer super output area     105
Constituency Code                  105
Quality                            104
User Type                          104
Last updated                       104
Postcode area                      104
Postcode district                  104
Police force             

In [30]:
(merged_post_code["In Use?"].isnull()==True).value_counts()

False    38982
True       104
Name: In Use?, dtype: int64

In [36]:
Incorrect_postal_codes = merged_post_code[merged_post_code["In Use?"].isnull()==True].index.tolist()

In [37]:
# Buradaki posta kodlarında harf hataları var. Mesela 0 yerine O yazılanlar vb.. farklı hatalar mevcut. 
# Bu nedenle düzenleme yapılması gerekiyor ancak her değerin teker teker kontrol edilmesi gerekiyor bu işlem çok zaman alacak 
# toplamda 104 satırı ilgilendiren bir durum söz konusu datanın geneline bir zararı olmayacağı düşüncesiyle bunları bu şekilde bırakıyorum.
for i in Incorrect_postal_codes:
    if i in satislar_pcode.index:
        print(satislar_pcode.iloc[i])

Postcode    DN15 0QU
Name: 742, dtype: object
Postcode    B71 8QT
Name: 1588, dtype: object
Postcode    B71 8QT
Name: 1592, dtype: object
Postcode    JE3 8FS
Name: 4474, dtype: object
Postcode    IM5 3BJ
Name: 5442, dtype: object
Postcode    JE2 7UD
Name: 30264, dtype: object
Postcode    GY4 6SR
Name: 34762, dtype: object
Postcode    LL16 9HF
Name: 39380, dtype: object
Postcode    LL16 9HF
Name: 55744, dtype: object
Postcode    BN42 2HR
Name: 56458, dtype: object
Postcode    B72 5BN
Name: 68524, dtype: object
Postcode    IM7 5AS
Name: 69970, dtype: object
Postcode    WV15 7AL
Name: 78800, dtype: object
Postcode    GY2 4EX
Name: 80364, dtype: object
Postcode    GY1 1GA
Name: 96192, dtype: object
Postcode    GY6 8BZ
Name: 98484, dtype: object
Postcode    NG15- 7QA
Name: 99352, dtype: object
Postcode    LL16 9HF
Name: 103606, dtype: object
Postcode    TQ9 0NH
Name: 112024, dtype: object
Postcode    JE3 7AS
Name: 122254, dtype: object
Postcode    GY1 1QJ
Name: 142380, dtype: object
Postcod

In [38]:
merged_post_code.head()

,Postcode,In Use?,Latitude,Longitude,Grid Ref,County,District,Ward,District Code,Ward Code,...,Quality,User Type,Last updated,Postcode area,Postcode district,Police force,Average Income,Travel To Work Area,ITL level 2,ITL level 3
0,OL13 8BT,Yes,53.727782,-2.195633,SD871257,Lancashire,Rossendale,Greensclough,E07000125,E05005321,...,1.0,0.0,2022-11-26,OL,OL13,Lancashire,35900.0,Blackburn,Lancashire,East Lancashire
1,DA1 5WW,Yes,51.448165,0.220005,TQ543744,Kent,Dartford,Town,E07000107,E05012409,...,1.0,0.0,2022-11-25,DA,DA1,Kent,52800.0,London,Kent,Kent Thames Gateway
2,W3 6TD,Yes,51.520727,-0.259227,TQ208816,Greater London,Ealing,North Acton,E09000009,E05013527,...,1.0,0.0,2022-11-26,W,W3,Metropolitan Police,51800.0,London,Outer London - West and North West,Ealing
3,BD23 2NP,Yes,53.956871,-2.017139,SD989511,North Yorkshire,Craven,Skipton West,E07000163,E05009669,...,1.0,0.0,2022-11-25,BD,BD23,North Yorkshire,38600.0,Skipton,North Yorkshire,North Yorkshire CC
4,NE13 7FJ,Yes,55.052837,-1.681323,NZ204731,Tyne and Wear,Newcastle upon Tyne,Castle,E08000021,E05011441,...,1.0,0.0,2022-11-25,NE,NE13,Northumbria,40100.0,Newcastle,"Northumberland, and Tyne and Wear",Tyneside


In [39]:
merged_post_code["Rural/urban"].value_counts(dropna=False)

Urban city and town                                        15610
Urban major conurbation                                    11351
Rural town and fringe                                       3499
Rural village                                               2036
Rural hamlet and isolated dwellings                         1366
Large urban area                                            1098
Urban minor conurbation                                      948
Other urban area                                             931
NaN                                                          553
Accessible rural area                                        490
Accessible small town                                        254
Rural town and fringe in a sparse setting                    207
Rural village in a sparse setting                            191
Rural hamlet and isolated dwellings in a sparse setting      156
Remote rural area                                            133
Urban city and town in a 

In [40]:
merged_post_code["Average Income"].mean()

43857.00267869731

In [41]:
merged_post_code.County.value_counts()

Greater London              5343
West Midlands               1490
Greater Manchester          1468
Essex                       1335
Kent                        1214
Hampshire                   1131
West Yorkshire              1100
Merseyside                   841
Surrey                       820
Lancashire                   813
Hertfordshire                754
Staffordshire                723
Devon                        721
Lincolnshire                 698
South Yorkshire              674
Cheshire                     671
North Yorkshire              652
Nottinghamshire              641
Leicestershire               614
Tyne and Wear                605
West Sussex                  602
County Durham                589
Berkshire                    573
Cambridgeshire               572
East Sussex                  567
Derbyshire                   562
Gloucestershire              547
Norfolk                      544
Somerset                     542
Buckinghamshire              525
Northampto

In [42]:
merged_post_code["User Type"].value_counts()

0.0    38878
1.0      104
Name: User Type, dtype: int64

In [44]:
merged_post_code.groupby("County").Households.mean()

County
Bedfordshire                31.581325
Berkshire                   30.925553
Bristol                     25.401826
Buckinghamshire             26.469957
Cambridgeshire              30.379921
Cheshire                    29.345178
City of London              31.500000
Clwyd                       25.291815
Cornwall                    25.275482
County Durham               28.423698
Cumbria                     30.135747
Derbyshire                  31.202020
Devon                       26.456731
Dorset                      27.217865
Dyfed                       23.505703
East Riding of Yorkshire    33.063830
East Sussex                 30.009328
Essex                       27.947761
Gloucestershire             26.889610
Greater London              35.083859
Greater Manchester          28.416792
Gwent                       28.143488
Gwynedd                     22.317568
Hampshire                   28.269347
Herefordshire               25.428571
Hertfordshire               26.886981
Isle 

In [45]:
merged_post_code.to_csv("post_code_analysis.csv", index = False)